## LCEL 인터페이스


사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. 

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다.
표준 인터페이스에는 다음이 포함됩니다.

- [`stream`](#stream): 응답의 청크를 스트리밍합니다.
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [6]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


LCEL 문법을 사용하여 chain 을 생성합니다.

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model | StrOutputParser()

In [5]:
chain

PromptTemplate(input_variables=['topic'], template='{topic} 에 대하여 3문장으로 설명해줘.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F6F7332250>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F6F712C750>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

## stream: 실시간 출력


이 함수는 `chain.stream` 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다. 

In [ ]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for token in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

## invoke: 호출


`chain` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [ ]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

## batch: 배치(단위 실행)


함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행합니다.

In [10]:
# 함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행합니다.
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
anwer = chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

In [14]:
anwer[1]

'인스타그램은 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼이다. 사용자들은 팔로우하고 좋아요를 누르며 소통하며 자신의 일상을 공유할 수 있다. 또한 인스타그램 스토리 기능을 통해 임시적인 콘텐츠를 공유할 수 있다.'

`max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

`config` 딕셔너리는 `max_concurrency` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [8]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 사용자와 자연스럽게 대화하며 다양한 주제에 대해 정보를 제공합니다. 이 모델은 텍스트 기반으로 작동하며, 사용자의 질문에 대답하거나 대화를 이어가는 데 사용됩니다. ChatGPT는 인간과의 상호작용을 향상시키는 데 도움이 되는 기술적 혁신의 한 예입니다.',
 'Instagram은 사진과 동영상을 공유하는 소셜 미디어 플랫폼으로, 사용자들은 자신의 일상이나 취향을 시각적으로 공유할 수 있습니다. 해시태그를 통해 관심사나 주제별로 다양한 콘텐츠를 찾을 수 있으며, 팔로워들과 소통하고 소셜 네트워킹을 할 수 있습니다. 인스타그램은 모바일 앱을 통해 간편하게 이용할 수 있으며, 광고 플랫폼으로도 활발히 이용되고 있습니다.',
 '멀티모달은 여러 가지 다양한 형태의 커뮤니케이션 방식을 결합하여 사용자에게 정보를 전달하는 방법이다. 이는 텍스트, 음성, 이미지, 동영상 등 여러 매체를 활용하여 사용자 경험을 향상시키고, 효율적인 소통을 도모하는 것을 의미한다. 멀티모달은 사용자에게 더욱 풍부한 정보를 제공함으로써 상호작용을 강화하고, 사용자의 이해도와 만족도를 높이는 데 도움을 준다.',
 '프로그래밍은 컴퓨터에게 실행할 작업을 지시하기 위해 사용되는 일련의 명령어를 작성하는 과정이다. 이러한 명령어는 프로그래밍 언어를 통해 작성되며, 문제 해결 능력과 논리적 사고가 필요하다. 프로그래밍을 통해 다양한 소프트웨어 및 애플리케이션을 개발할 수 있으며, 현대 사회에서 매우 중요한 역할을 한다.',
 '머신러닝은 컴퓨터 시스템이 데이터를 학습하고 패턴을 식별하여 예측하도록 하는 인공지능 기술이다. 이를 통해 컴퓨터는 경험을 토대로 문제를 해결하고 새로운 정보를 학습할 수 있다. 머신러닝은 다양한 분야에서 활용되어 효율적인 의사결정과 예측을 가능하게 한다.']

## async stream: 비동기 스트림


함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print` 함수를 통해 메시지의 내용(`s.content`)을 즉시 출력합니다. `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.


In [15]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

YouTube는 동영상 공유 플랫폼으로, 사용자들은 다양한 주제의 동영상을 업로드하고 시청할 수 있다. 유튜버들은 채널을 통해 자신의 콘텐츠를 공유하고 구독자들과 소통할 수 있다. 광고 수익을 얻을 수 있는 등 다양한 방법으로 수익을 창출할 수 있는 인기 있는 플랫폼이다.

## async invoke: 비동기 호출


`chain` 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 `topic`이라는 키와 `NVDA`(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.


In [16]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic": "NVDA"})

In [17]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다립니다.
await my_process

'NVDA는 엔비디아 코퍼레이션의 주식 코드로, 미국의 반도체 기업이다. 엔비디아는 그래픽 처리 장치(GPU)와 시스템 온 칩(SoC)을 생산하여 주로 게임 산업과 인공지능 분야에서 사용된다. NVDA 주식은 기술 기업 투자자들 사이에서 인기가 높은 주식 중 하나이다.'

## async batch: 비동기 배치


함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 `chain` 객체의 `abatch` 메서드를 사용하여 `topic` 에 대한 작업을 비동기적으로 처리하고 있습니다.

`await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.


In [19]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

C:\Users\jykim\AppData\Local\Temp\ipykernel_3568\1297528847.py:2: RuntimeWarning: coroutine 'RunnableSequence.abatch' was never awaited
  my_abatch_process = chain.abatch(


In [20]:
# 비동기로 처리되는 일괄 처리 프로세스가 완료될 때까지 기다립니다.
await my_abatch_process

['YouTube는 온라인 동영상 공유 플랫폼으로, 사용자들이 자신의 동영상을 업로드하고 시청할 수 있는 서비스이다. 다양한 콘텐츠를 제공하며, 누구나 쉽게 동영상을 만들어 공유할 수 있어 많은 사람들에게 사랑받고 있다. 유명한 크리에이터들의 채널을 구독하거나 원하는 주제의 동영상을 검색하여 시청할 수 있다.',
 'Instagram은 사진과 동영상을 공유하고 소셜 네트워크 서비스를 제공하는 앱이다. 사용자들은 다양한 필터와 효과를 이용하여 자신의 콘텐츠를 꾸밀 수 있으며, 팔로워와 소통하며 새로운 친구들을 만들 수 있다. 또한 인플루언서와 기업들이 광고 및 마케팅 활동을 펼칠 수 있는 플랫폼으로도 활용된다.',
 '페이스북은 세계에서 가장 인기 있는 소셜 네트워크 서비스 중 하나이다. 사용자들은 친구와 가족과 소통하고, 사진 및 동영상을 공유할 수 있다. 또한 광고주들이 제품 및 서비스를 홍보하는 데에도 이용된다.']

## Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다.
예를 들어, `RunnableParallel`을 사용할 때, 각 요소를 병렬로 실행합니다.


`langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.

`ChatPromptTemplate.from_template` 메서드를 사용하여 주어진 `country`에 대한 **수도** 와 **면적** 을 구하는 두 개의 체인(`chain1`, `chain2`)을 만듭니다.

이 체인들은 각각 `model`과 파이프(`|`) 연산자를 통해 연결됩니다. 마지막으로, `RunnableParallel` 클래스를 사용하여 이 두 체인을 `capital`와 `area`이라는 키로 결합하여 동시에 실행할 수 있는 `combined` 객체를 생성합니다.


In [21]:
#`langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

`chain1.invoke()` 함수는 `chain1` 객체의 `invoke` 메서드를 호출합니다.

이때, `country`이라는 키에 `대한민국`라는 값을 가진 딕셔너리를 인자로 전달합니다.


In [22]:
# chain1 를 실행합니다.
chain1.invoke({"country": "대한민국"})

'대한민국의 수도는 서울이야.'

이번에는 `chain2.invoke()` 를 호출합니다. `country` 키에 다른 국가인 `미국` 을 전달합니다.


In [23]:
# chain2 를 실행합니다.
chain2.invoke({"country": "미국"})

'미국의 면적은 약 9,833,520 km² 입니다.'

`combined` 객체의 `invoke` 메서드는 주어진 `country`에 대한 처리를 수행합니다.

이 예제에서는 `대한민국`라는 주제를 `invoke` 메서드에 전달하여 실행합니다.


In [24]:
# 병렬 실행 체인을 실행합니다.
combined.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 총 면적은 약 100,363km² 입니다.'}

### 배치에서의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다.
배치와 병렬 처리를 사용해 보도록 합시다.


`chain1.batch` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.


In [27]:
# 배치 처리를 수행합니다.
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 수도는 서울입니다.', '미국의 수도는 워싱턴 D.C.입니다.']

`chain2.batch` 함수는 여러 개의 딕셔너리를 리스트 형태로 받아, 일괄 처리(batch)를 수행합니다.

이 예시에서는 `대한민국`와 `미국`라는 두 가지 국가에 대한 처리를 요청합니다.


In [28]:
# 배치 처리를 수행합니다.
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

['대한민국의 면적은 약 100,363.4 제곱킬로미터입니다.', '미국의 면적은 9,826,675 km² 입니다.']

`combined.batch` 함수는 주어진 데이터를 배치로 처리하는 데 사용됩니다. 이 예시에서는 두 개의 딕셔너리 객체를 포함하는 리스트를 인자로 받아 각각 `대한민국`와 `미국` 두 나라에 대한 데이터를 배치 처리합니다.


In [29]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': '대한민국의 수도는 서울이다.', 'area': '대한민국의 총 면적은 약 100,363 제곱킬로미터입니다.'},
 {'capital': '미국의 수도는 워싱턴 D.C.입니다.', 'area': '미국의 면적은 약 9,826만 제곱 킬로미터입니다.'}]